<a href="https://colab.research.google.com/github/Punctuality/Big_Data_Infrastructure_ITMO_2023_Lab_1/blob/development/notebooks/Fake_News_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>